# 1. Data Generator
- Raw Data를 읽어옴
- 여기서 만들어진 데이터는 모델의 입력으로 들어감

In [1]:
import os
import numpy as np
import librosa
from tensorflow.keras.utils import Sequence

In [2]:
class RawForVAEGenerator(Sequence):
    def __init__(self, source, wav_dir, files, sourNum='s1', batch_size=10, shuffle=True):
        self.source = source
        self.wav_dir = wav_dir
        self.files = files
        self.sourNum = sourNum
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
        self.sample_rate = 8000
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.source))
        
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __audioread__(self, path, offset=0.0, duration=None, sample_rate=16000):
        signal = librosa.load(path, sr=self.sample_rate, mono=False, offset=offset, duration=duration)

        return signal[0]
    
    def __padding__(self, data):
        n_batch = len(data)
        max_len = max([d.shape[0] for d in data])
        extrapadding = int(np.ceil(max_len / self.sample_rate) * self.sample_rate)
        pad = np.zeros((n_batch, extrapadding))
        
        for i in range(n_batch):
            pad[i, :data[i].shape[0]] = data[i]
        
        return np.expand_dims(pad, -1)
        
    def __data_generation__(self, source_list):
        wav_list = []
        for name in source_list:
            name = name.strip('\n')
            
            s_wav_name = self.wav_dir + self.files + '/' + self.sourNum + '/' + name
            
            # ------- AUDIO READ -------
            s_wav = (self.__audioread__(s_wav_name,  offset=0.0, duration=None, sample_rate=self.sample_rate))
            # --------------------------
            
            # ------- PADDING -------
#             pad_len = max(len(samples1),len(samples2))
#             pad_s1 = np.concatenate([s1_wav, np.zeros([pad_len - len(s1_wav)])])
            
#             extrapadding = ceil(len(pad_s1) / sample_rate) * sample_rate - len(pad_s1)
#             pad_s1 = np.concatenate([pad_s1, np.zeros([extrapadding - len(pad_s1)])])
#             pad_s2 = np.concatenate([s2_wav, np.zeros([extrapadding - len(s2_wav)])])
            # -----------------------
            
            wav_list.append(s_wav)
        
        return wav_list, wav_list, source_list
            
    
    def __len__(self):
        return int(np.floor(len(self.source) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        source_list = [self.source[k] for k in indexes]
        
        if self.files is not 'tt':
            sour, labels, _ = self.__data_generation__(source_list)
            
            # Get Lengths(K value of each batch)
            lengths = np.array([m.shape[0] for m in sour])
            exp = np.expand_dims(lengths, 1)
            exp = np.expand_dims(exp, -1) # [Batch, 1, 1] (length)
            
            # Padding
            sour_pad = self.__padding__(sour) # [Batch, Time_step, Dimension(=1)]
            label_pad = self.__padding__(labels) # [Batch, Time_step, Dimension(=1)]
            
            return sour_pad, np.concatenate([label_pad, exp], axis=1)
        else:
            sour, labels, name = self.__data_generation__(source_list)
            
            # Get Lengths(K value of each batch)
            lengths = np.array([m.shape[0] for m in sour])
            exp = np.expand_dims(lengths, 1)
            exp = np.expand_dims(exp, -1) # [Batch, 1, 1] (length)
            
            # Padding
            sour_pad = self.__padding__(sour) # [Batch, Time_step, Dimension(=1)]
            
            return sour_pad, exp, name

## Data를 어떻게 읽는지에 대한 부분

In [3]:
WAV_DIR = './mycode/wsj0_2mix/use_this/'
LIST_DIR = './mycode/wsj0_2mix/use_this/lists/'

In [4]:
# Directory List file create

wav_dir = WAV_DIR
output_lst = LIST_DIR

for folder in ['tr', 'cv', 'tt']:
    wav_files = os.listdir(wav_dir + folder + '/mix')
    output_lst_files = output_lst + folder + '_wav.lst'
    with open(output_lst_files, 'w') as f:
        for file in wav_files:
            f.write(file + "\n")

print("Generate wav file to .lst done!")

Generate wav file to .lst done!


In [45]:
batch_size = 2

train_dataset = 0
valid_dataset = 0
test_dataset = 0

name_list = []
for files in ['tr', 'cv', 'tt']:
    # --- Lead lst file ---""
    output_lst_files = LIST_DIR + files + '_wav.lst'
    fid = open(output_lst_files, 'r')
    lines = fid.readlines()
    fid.close()
    # ---------------------
    
    if files == 'tr':
        train_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
    elif files == 'cv':
        valid_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
    else:
        test_batch = 1
        test_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', test_batch)

# 2. Building VQ-VAE model with Gumbel Softmax

In [46]:
import threading
from scipy.io.wavfile import write as wav_write
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from tensorflow.keras import backend as Kb
import numpy as np
import pandas as pd
from importlib import reload
import time
from tensorflow.keras.models import Model, Sequential, load_model

In [47]:
def mkdir_p(path):
    """ Creates a path recursively without throwing an error if it already exists
    :param path: path to create
    :return: None
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [48]:
mkdir_p('./CKPT/') # model check point 폴더 만드는 코드
filepath = "./CKPT/CKP_ep_{epoch:d}__loss_{val_loss:.5f}_.h5"

In [49]:
initial_learning_rate = 0.001

# learning rate를 점점 줄이는 부분
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

# validation loss에 대해서 좋은 것만 저장됨
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min'
)

# early stop 하는 부분인데, validation loss에 대해서 제일 좋은 모델이 저장됨
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=50, restore_best_weights=True
)

In [50]:
class GumbelSoftmax(layers.Layer):
    def __init__(self, temperature=0.5, hard=False, name = 'gumbel_softmax',**kwargs):
        super(GumbelSoftmax, self).__init__(name=name, **kwargs)
        
        self.temperature = temperature
        self.hard = hard
    
    def sample_gumbel(self, shape, eps=1e-20): 
        """Sample from Gumbel(0, 1)"""
        U = tf.random.uniform(shape,minval=0,maxval=1)
        
        return -tf.math.log(-tf.math.log(U + eps) + eps)

    def gumbel_softmax_sample(self, logits, temperature): 
        """ Draw a sample from the Gumbel-Softmax distribution"""
        y = logits + self.sample_gumbel(tf.shape(logits))
        
        return tf.nn.softmax(y / temperature)

    def call(self, inputs):
        y = self.gumbel_softmax_sample(inputs, self.temperature)
        
        if self.hard:
            y_hard = tf.cast(tf.equal(y, tf.math.reduce_max(y, 2, keepdims=True)), y.dtype)
            y = tf.stop_gradient(y_hard - y) + y
        
        return y


class Encoder(layers.Layer):
    def __init__(self, latent_dim, name = 'encoder',**kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        
        self.conv1d_1 = layers.Conv1D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu')
        self.conv1d_2 = layers.Conv1D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu')
        self.conv1d_3 = layers.Conv1D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu')
        self.logit = layers.Conv1D(filters=latent_dim, kernel_size=3, strides=2, activation='relu', padding='same')
    
    def call(self, inputs):
        x = self.conv1d_1(inputs)
        x = self.conv1d_2(x)
        x = self.conv1d_3(x)
        logit = self.logit(x)
        
        return logit


class Decoder(layers.Layer):
    def __init__(self, latent_dim, name = 'decoder',**kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        
        self.trans_conv1d_1 = layers.Conv1DTranspose(filters=256, kernel_size=3, strides=2, activation='relu', padding='same')
        self.trans_conv1d_2 = layers.Conv1DTranspose(filters=128, kernel_size=3, strides=2, activation='relu', padding='same')
        self.trans_conv1d_3 = layers.Conv1DTranspose(filters=64, kernel_size=3, strides=2, activation='relu', padding='same')
        self.logit = layers.Conv1DTranspose(filters=1, kernel_size=3, strides=2, padding='same', activation=None)
    
    def call(self, inputs):
        x = self.trans_conv1d_1(inputs)
        x = self.trans_conv1d_2(x)
        x = self.trans_conv1d_3(x)
        logit = self.logit(x)
        
        return logit

In [55]:
# Custom Metric Si-sdr

class SiSdr(keras.metrics.Metric):
    def __init__(self, name="Si-sdr", **kwargs):
        super(SiSdr, self).__init__(name=name, **kwargs)
        self.sdr = self.add_weight(name="sdr", initializer="zeros")
        self.count = self.add_weight(name="cnt", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        ori_length = tf.shape(y_true)[1]
        
        # Label & Length divide
        labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 1]
        lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]
        
        # Check sequence length
        batch_size = tf.shape(labels)[0]
        label_size = tf.shape(labels)[1]
        pred_size = tf.shape(y_pred)[1]
        feature_size = tf.shape(labels)[-1]
        
        # Change sequence length
        if label_size < pred_size:
            y_pred = tf.slice(y_pred, [0, 0, 0], [-1, label_size, -1])
        elif label_size > pred_size:
            labels = tf.slice(labels, [0, 0, 0], [-1, pred_size, -1])

        # SI-SDR
        target = tf.linalg.matmul(y_pred, labels, transpose_a=True) * labels / tf.expand_dims(tf.experimental.numpy.square(tf.norm(labels, axis=1)), axis=-1)
        noise = y_pred - target
        values = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
        
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.sdr.assign_add(tf.reduce_sum(values))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.sdr / self.count

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.sdr.assign(0.0)
        self.count.assign(0.0)

In [56]:
# Custom loss

# Custom mse
def custom_mse(y_true, y_pred):
    ori_length = tf.shape(y_true)[1]

    # Label & Length divide
    labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 129]
    lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]

    loss = tf.reduce_sum(tf.pow(y_pred - labels, 2))

    return loss


# Custom si-sdr loss
def custom_sisdr_loss(y_true, y_pred):
    ori_length = tf.shape(y_true)[1]

    # Label & Length divide
    labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 1]
    lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]

    target = tf.linalg.matmul(y_pred, labels, transpose_a=True) * labels / tf.expand_dims(tf.experimental.numpy.square(tf.norm(labels, axis=1)), axis=-1)
    noise = y_pred - target
    si_sdr = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
    si_sdr = tf.reduce_mean(si_sdr) * -1

    return si_sdr

In [57]:
class Vq_vae(keras.Model):
    def __init__(self, latent_dim, gumbel_hard=False, name='vqvae', **kwargs):
        super(Vq_vae, self).__init__(name=name, **kwargs)
        
        self.latent_dim = latent_dim
        self.softmax = layers.Softmax(-1)
        
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)
        self.gumbel = GumbelSoftmax(hard=gumbel_hard)
        
    def call(self, inputs, load=False):
        if load:
            inputs = layers.Input(shape=(None, 1))
        
        
        encode = self.encoder(inputs)
        gumbel = self.gumbel(encode)
        decode = self.decoder(gumbel)
        
        # ------------------ KL loss ------------------
        qy = self.softmax(encode)
        log_qy = tf.math.log(qy + 1e-20)
        log_uniform = qy * (log_qy - tf.math.log(1.0 / self.latent_dim))
        kl_loss = tf.reduce_mean(log_uniform)
        # ---------------------------------------------
        
        self.add_loss(kl_loss)
        
        return decode

In [58]:
latent_size = 512
epoch = 300
BATCH_SIZE = 2

strategy = tf.distribute.MirroredStrategy(['cpu:0'])
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model_path = './CKPT/CKP_ep_30__loss_158.95885_.h5'
    
    loss_fun = custom_mse
#     loss_fun = custom_sisdr_loss
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=False)

    optimizer = keras.optimizers.Adam(learning_rate=1e-3)
    vq_vae.compile(optimizer, loss=loss_fun, metrics=[SiSdr()])
    
    vq_vae(0, True)
    vq_vae.summary()
    
    # 사용 안할 때는 load_model 주석 처리 하자
#     vq_vae.load_weights(model_path)
    # ----------------------------------------
    
    tf.executing_eagerly()

history = vq_vae.fit(
    train_dataset,
    epochs=epoch,
    validation_data=valid_dataset,
    shuffle=True,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
장치의 수: 1
Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_7 (Softmax)          (None, None, 512)         0         
_________________________________________________________________
encoder (Encoder)            (None, None, 512)         517248    
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           516737    
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 512)         0         
Total params: 1,033,985
Trainable params: 1,033,985
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
2/2 [==============================] - 5s 2s/step - loss: 631.1898 - Si-sdr: -54.7975 - val_loss: 629.3751 - val_Si-sdr: -55.9

2/2 [==============================] - 1s 635ms/step - loss: 551.9090 - Si-sdr: -8.4797 - val_loss: 539.4868 - val_Si-sdr: -7.9204

Epoch 00030: val_loss improved from 558.94312 to 539.48676, saving model to ./CKPT\CKP_ep_30__loss_539.48676_.h5
Epoch 31/300
2/2 [==============================] - 1s 761ms/step - loss: 529.1852 - Si-sdr: -7.4093 - val_loss: 500.5460 - val_Si-sdr: -5.8928

Epoch 00031: val_loss improved from 539.48676 to 500.54599, saving model to ./CKPT\CKP_ep_31__loss_500.54599_.h5
Epoch 32/300
2/2 [==============================] - 1s 659ms/step - loss: 495.7227 - Si-sdr: -5.5905 - val_loss: 470.5332 - val_Si-sdr: -4.2987

Epoch 00032: val_loss improved from 500.54599 to 470.53323, saving model to ./CKPT\CKP_ep_32__loss_470.53323_.h5
Epoch 33/300
2/2 [==============================] - 1s 641ms/step - loss: 461.1436 - Si-sdr: -4.0021 - val_loss: 434.5842 - val_Si-sdr: -3.4026

Epoch 00033: val_loss improved from 470.53323 to 434.58417, saving model to ./CKPT\CKP_ep_33__

2/2 [==============================] - 1s 642ms/step - loss: 166.6244 - Si-sdr: 4.4220 - val_loss: 164.0025 - val_Si-sdr: 4.4962

Epoch 00062: val_loss improved from 167.12708 to 164.00252, saving model to ./CKPT\CKP_ep_62__loss_164.00252_.h5
Epoch 63/300
2/2 [==============================] - 1s 796ms/step - loss: 164.3413 - Si-sdr: 4.4934 - val_loss: 160.7601 - val_Si-sdr: 4.6202

Epoch 00063: val_loss improved from 164.00252 to 160.76009, saving model to ./CKPT\CKP_ep_63__loss_160.76009_.h5
Epoch 64/300
2/2 [==============================] - 1s 704ms/step - loss: 161.4836 - Si-sdr: 4.5870 - val_loss: 160.7765 - val_Si-sdr: 4.6097

Epoch 00064: val_loss did not improve from 160.76009
Epoch 65/300
2/2 [==============================] - 1s 669ms/step - loss: 159.2918 - Si-sdr: 4.6650 - val_loss: 158.5458 - val_Si-sdr: 4.7085

Epoch 00065: val_loss improved from 160.76009 to 158.54579, saving model to ./CKPT\CKP_ep_65__loss_158.54579_.h5
Epoch 66/300
2/2 [==============================]

2/2 [==============================] - 1s 678ms/step - loss: 119.1384 - Si-sdr: 6.2108 - val_loss: 120.0763 - val_Si-sdr: 6.1745

Epoch 00097: val_loss did not improve from 117.83438
Epoch 98/300
2/2 [==============================] - 1s 681ms/step - loss: 118.4677 - Si-sdr: 6.2380 - val_loss: 117.4224 - val_Si-sdr: 6.2808

Epoch 00098: val_loss improved from 117.83438 to 117.42240, saving model to ./CKPT\CKP_ep_98__loss_117.42240_.h5
Epoch 99/300
2/2 [==============================] - 1s 748ms/step - loss: 118.0248 - Si-sdr: 6.2593 - val_loss: 117.3988 - val_Si-sdr: 6.2730

Epoch 00099: val_loss improved from 117.42240 to 117.39876, saving model to ./CKPT\CKP_ep_99__loss_117.39876_.h5
Epoch 100/300
2/2 [==============================] - 1s 678ms/step - loss: 118.2218 - Si-sdr: 6.2444 - val_loss: 117.0382 - val_Si-sdr: 6.3110

Epoch 00100: val_loss improved from 117.39876 to 117.03818, saving model to ./CKPT\CKP_ep_100__loss_117.03818_.h5
Epoch 101/300
2/2 [============================

2/2 [==============================] - 1s 743ms/step - loss: 100.2040 - Si-sdr: 7.0830 - val_loss: 101.3407 - val_Si-sdr: 6.9951

Epoch 00133: val_loss did not improve from 99.11692
Epoch 134/300
2/2 [==============================] - 1s 627ms/step - loss: 102.2211 - Si-sdr: 6.9868 - val_loss: 100.4242 - val_Si-sdr: 7.0739

Epoch 00134: val_loss did not improve from 99.11692
Epoch 135/300
2/2 [==============================] - 1s 752ms/step - loss: 102.9584 - Si-sdr: 6.9218 - val_loss: 102.0878 - val_Si-sdr: 6.9714

Epoch 00135: val_loss did not improve from 99.11692
Epoch 136/300
2/2 [==============================] - 1s 674ms/step - loss: 100.1880 - Si-sdr: 7.0627 - val_loss: 102.6768 - val_Si-sdr: 6.9483

Epoch 00136: val_loss did not improve from 99.11692
Epoch 137/300
2/2 [==============================] - 1s 730ms/step - loss: 100.6889 - Si-sdr: 7.0582 - val_loss: 100.5697 - val_Si-sdr: 7.0616

Epoch 00137: val_loss did not improve from 99.11692
Epoch 138/300
2/2 [===============

2/2 [==============================] - 1s 745ms/step - loss: 92.7899 - Si-sdr: 7.4601 - val_loss: 94.2262 - val_Si-sdr: 7.3797

Epoch 00172: val_loss did not improve from 90.35068
Epoch 173/300
2/2 [==============================] - 1s 737ms/step - loss: 94.8795 - Si-sdr: 7.3511 - val_loss: 92.1942 - val_Si-sdr: 7.4782

Epoch 00173: val_loss did not improve from 90.35068
Epoch 174/300
2/2 [==============================] - 1s 629ms/step - loss: 92.6809 - Si-sdr: 7.4673 - val_loss: 91.9366 - val_Si-sdr: 7.4726

Epoch 00174: val_loss did not improve from 90.35068
Epoch 175/300
2/2 [==============================] - 1s 640ms/step - loss: 94.0505 - Si-sdr: 7.3596 - val_loss: 93.7140 - val_Si-sdr: 7.4128

Epoch 00175: val_loss did not improve from 90.35068
Epoch 176/300
2/2 [==============================] - 1s 657ms/step - loss: 92.4946 - Si-sdr: 7.4650 - val_loss: 93.0159 - val_Si-sdr: 7.4488

Epoch 00176: val_loss did not improve from 90.35068
Epoch 177/300
2/2 [=========================

2/2 [==============================] - 1s 745ms/step - loss: 88.6839 - Si-sdr: 7.6740 - val_loss: 87.3058 - val_Si-sdr: 7.7204

Epoch 00212: val_loss did not improve from 86.93803
Epoch 213/300
2/2 [==============================] - 1s 751ms/step - loss: 89.0576 - Si-sdr: 7.6407 - val_loss: 87.3448 - val_Si-sdr: 7.7466

Epoch 00213: val_loss did not improve from 86.93803
Epoch 214/300
2/2 [==============================] - 1s 669ms/step - loss: 88.7600 - Si-sdr: 7.6534 - val_loss: 88.6251 - val_Si-sdr: 7.6822

Epoch 00214: val_loss did not improve from 86.93803
Epoch 215/300
2/2 [==============================] - 1s 661ms/step - loss: 90.2199 - Si-sdr: 7.6079 - val_loss: 88.5384 - val_Si-sdr: 7.6778

Epoch 00215: val_loss did not improve from 86.93803
Epoch 216/300
2/2 [==============================] - 1s 747ms/step - loss: 86.9065 - Si-sdr: 7.7904 - val_loss: 86.2196 - val_Si-sdr: 7.8146

Epoch 00216: val_loss improved from 86.93803 to 86.21959, saving model to ./CKPT\CKP_ep_216__los

2/2 [==============================] - 1s 625ms/step - loss: 84.6828 - Si-sdr: 7.9043 - val_loss: 84.0106 - val_Si-sdr: 7.9372

Epoch 00253: val_loss did not improve from 83.93993
Epoch 254/300
2/2 [==============================] - 1s 643ms/step - loss: 84.4676 - Si-sdr: 7.9095 - val_loss: 84.4423 - val_Si-sdr: 7.9149

Epoch 00254: val_loss did not improve from 83.93993
Epoch 255/300
2/2 [==============================] - 1s 740ms/step - loss: 83.7947 - Si-sdr: 7.9508 - val_loss: 84.0053 - val_Si-sdr: 7.9387

Epoch 00255: val_loss did not improve from 83.93993
Epoch 256/300
2/2 [==============================] - 1s 748ms/step - loss: 82.9870 - Si-sdr: 7.9975 - val_loss: 83.6136 - val_Si-sdr: 7.9546

Epoch 00256: val_loss improved from 83.93993 to 83.61360, saving model to ./CKPT\CKP_ep_256__loss_83.61360_.h5
Epoch 257/300
2/2 [==============================] - 1s 668ms/step - loss: 83.7990 - Si-sdr: 7.9379 - val_loss: 83.5835 - val_Si-sdr: 7.9741

Epoch 00257: val_loss improved from 8

2/2 [==============================] - 1s 636ms/step - loss: 81.1681 - Si-sdr: 8.1159 - val_loss: 80.6442 - val_Si-sdr: 8.1516

Epoch 00293: val_loss did not improve from 80.04192
Epoch 294/300
2/2 [==============================] - 1s 745ms/step - loss: 80.6328 - Si-sdr: 8.1648 - val_loss: 80.0329 - val_Si-sdr: 8.1891

Epoch 00294: val_loss improved from 80.04192 to 80.03293, saving model to ./CKPT\CKP_ep_294__loss_80.03293_.h5
Epoch 295/300
2/2 [==============================] - 1s 658ms/step - loss: 81.2953 - Si-sdr: 8.1300 - val_loss: 80.1810 - val_Si-sdr: 8.1770

Epoch 00295: val_loss did not improve from 80.03293
Epoch 296/300
2/2 [==============================] - 1s 660ms/step - loss: 80.2740 - Si-sdr: 8.1797 - val_loss: 80.3847 - val_Si-sdr: 8.1620

Epoch 00296: val_loss did not improve from 80.03293
Epoch 297/300
2/2 [==============================] - 1s 742ms/step - loss: 80.1794 - Si-sdr: 8.2024 - val_loss: 80.3840 - val_Si-sdr: 8.1725

Epoch 00297: val_loss did not improve

## 2.2. Encoder 부르는 방법, Decoder에 값 넣는 방법

In [50]:
latent_size = 512
epoch = 200
BATCH_SIZE = 2

strategy = tf.distribute.MirroredStrategy(['cpu:0'])
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model_path = './CKPT/CKP_ep_283__loss_141.77045_.h5'
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=False)
    vq_vae(0, True)
    vq_vae.summary()
    
    vq_vae.load_weights(model_path)
    
    # 이렇게 하면, transforer의 input으로 들어가는 one-hot 형식의 값을 얻을 수 있음
    for inputs, label in train_dataset:
        encode = vq_vae.encoder(inputs).numpy()
        encode_onehot = tf.cast(tf.equal(encode, tf.math.reduce_max(encode, 2, keepdims=True)), encode.dtype)
    
    # 이렇게 하면, transformer의 output을 vq-vae의 decoder 입력으로 넣을 수 있음
    for inputs, label in train_dataset:
        encode = vq_vae.encoder(inputs).numpy()
        encode_onehot = tf.cast(tf.equal(encode, tf.math.reduce_max(encode, 2, keepdims=True)), encode.dtype)
        
        # 이렇게 이전 layer의 출렫을 넣으면 됨
        decode = vq_vae.decoder(encode_onehot).numpy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
장치의 수: 1
Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_25 (Softmax)         (None, None, 512)         0         
_________________________________________________________________
encoder (Encoder)            (None, None, 512)         517248    
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           516737    
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 512)         0         
Total params: 1,033,985
Trainable params: 1,033,985
Non-trainable params: 0
_________________________________________________________________


# 3. Test Model

In [51]:
def mkdir_p(path):
    """ Creates a path recursively without throwing an error if it already exists
    :param path: path to create
    :return: None
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [52]:
mkdir_p('./test_wav/') # Result wav 폴더 만드는 코드

In [53]:
def audiowrite(data, path, samplerate=16000, normalize=False, threaded=True):
    """ Write the audio data ``data`` to the wav file ``path``
    The file can be written in a threaded mode. In this case, the writing
    process will be started at a separate thread. Consequently, the file will
    not be written when this function exits.
    :param data: A numpy array with the audio data
    :param path: The wav file the data should be written to
    :param samplerate: Samplerate of the audio data
    :param normalize: Normalize the audio first so that the values are within
        the range of [INTMIN, INTMAX]. E.g. no clipping occurs
    :param threaded: If true, the write process will be started as a separate
        thread
    :return: The number of clipped samples
    """
    data = data.copy()
    int16_max = np.iinfo(np.int16).max
    int16_min = np.iinfo(np.int16).min

    if normalize:
        if not data.dtype.kind == 'f':
            data = data.astype(np.float)
        data /= np.max(np.abs(data))

    if data.dtype.kind == 'f':
        data *= int16_max

    sample_to_clip = np.sum(data > int16_max)
    if sample_to_clip > 0:
        print('Warning, clipping {} samples'.format(sample_to_clip))
    data = np.clip(data, int16_min, int16_max)
    data = data.astype(np.int16)

    if threaded:
        threading.Thread(target=wav_write, args=(path, samplerate, data)).start()
    else:
        wav_write(path, samplerate, data)

    return sample_to_clip

In [57]:
with tf.device('/cpu:0'):
    latent_size = 512
    sample_rate = 8000
    model_path = './CKPT/CKP_ep_283__loss_141.77045_.h5'
    
    vq_vae = Vq_vae(latent_size, gumbel_hard=True)
    vq_vae(0, True)
    vq_vae.summary()
    vq_vae.load_weights(model_path)

    for batch in test_dataset:
        input_batch, length_batch, name = batch

        result = vq_vae.predict(input_batch)
        
        wav_name = './test_wav/' + name[0][:-5] + '_s1.wav'
        audiowrite(result[0], wav_name, sample_rate, True, True)

Model: "vqvae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
softmax_29 (Softmax)         (None, None, 512)         0         
_________________________________________________________________
encoder (Encoder)            (None, None, 512)         517248    
_________________________________________________________________
decoder (Decoder)            (None, None, 1)           516737    
_________________________________________________________________
gumbel_softmax (GumbelSoftma (None, None, 512)         0         
Total params: 1,033,985
Trainable params: 1,033,985
Non-trainable params: 0
_________________________________________________________________


# 여기 밑에는 연습장임

In [16]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding

model = Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(None, 1)))
model.add(layers.Conv1D(filters=4, kernel_size=3, padding='same'))

input_array = np.random.randn(2, 10, 1)
input_array2 = np.random.randn(2, 9, 1)
with tf.device('/cpu:0'):
    model.compile('rmsprop', 'mse')

    output_array = model.predict(input_array)
    output_array2 = model.predict(input_array2)

In [22]:
tf.cast(tf.shape(output_array)[0], tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [17]:
print(output_array)
print(output_array.shape)
print(output_array2.shape)

[[[-2.62064394e-02  8.17203820e-02  3.24754268e-02 -1.02852143e-01]
  [-1.49157234e-02 -1.95800625e-02 -3.46802175e-02  5.72501123e-03]
  [-7.80363381e-02 -6.07307628e-02 -9.69931781e-02 -5.16347066e-02]
  [-2.81637251e-01  9.96127948e-02 -3.17231864e-02  3.74453515e-03]
  [-1.18284672e-01  8.46180767e-02 -1.95280939e-01  2.93599963e-02]
  [-3.21779072e-01  1.22240782e-02 -1.09718263e-01  7.65489265e-02]
  [-3.70774984e-01  2.58951426e-01 -1.33752108e-01 -1.99191421e-02]
  [-3.68880987e-01 -1.90456361e-02 -1.05885044e-01 -5.90537339e-02]
  [-3.16750824e-01 -3.97674739e-04 -1.01323225e-01 -1.01796679e-01]
  [ 3.26998197e-02  1.05254844e-01  2.80397180e-02  1.57616232e-02]]

 [[-1.11196786e-01  1.94959119e-02 -1.16245545e-01 -3.68264988e-02]
  [-5.72053492e-02 -1.78234074e-02 -5.17670363e-02 -2.48496141e-02]
  [-1.82820857e-01  6.84121251e-02 -5.39908707e-02 -5.49857728e-02]
  [-1.42236158e-01  1.38846397e-01 -5.72850741e-02 -4.44535911e-03]
  [-2.06366882e-01  4.27138396e-02 -1.56833917

In [358]:
batch_size = tf.shape(output_array)[0]
array1_size = tf.shape(output_array)[1]
array2_size = tf.shape(output_array2)[1]
feature_size = tf.shape(output_array)[-1]

if array1_size < array2_size:
#     append_size = array1_size - array2_size
#     append_zeros = tf.zeros([batch_size, append_size, feature_size])
#     append_zeros = tf.Variable(initial_value=tf.zeros((batch_size, append_size, feature_size)))
#     output_array2 = tf.concat([output_array2, append_zeros], axis=1)
    output_array2 = tf.slice(output_array2, [0, 0, 0], [-1, array1_size, -1])
elif array1_size > array2_size:
#     append_size = array2_size - array1_size
#     append_zeros = tf.zeros([batch_size, append_size, feature_size])
#     append_zeros = tf.Variable(initial_value=tf.zeros((batch_size, append_size, feature_size)))
#     output_array = tf.concat([output_array, append_zeros], axis=1)
    output_array = tf.slice(output_array, [0, 0, 0], [-1, array2_size, -1])

print(output_array.shape)
print(output_array2.shape)
# output_array0 = output_array[1]
# output_array20 = output_array2[1]
# target = np.sum(output_array20 * output_array0) * output_array0 / np.square(np.linalg.norm(output_array0, ord=2))
# noise = output_array20 - target
# npnp = 10 * np.log10(np.square(np.linalg.norm(target, ord=2)) / np.square(np.linalg.norm(noise, ord=2)))
# print(npnp)

# target = tf.linalg.matmul(output_array2, output_array, transpose_a=True) * output_array / tf.expand_dims(tf.experimental.numpy.square(tf.norm(output_array, axis=1)), axis=-1)
# noise = output_array2 - target
# si_sdr = 10 * tf.experimental.numpy.log10(tf.experimental.numpy.square(tf.norm(target, axis=1)) / tf.experimental.numpy.square(tf.norm(noise, axis=1)))
# si_sdr = tf.reduce_mean(si_sdr)
# print(si_sdr)

(2, 9, 1)
(2, 9, 1)


In [79]:
tf.cast(tf.equal(output_array, tf.math.reduce_max(output_array, 2, keepdims=True)), output_array.dtype)

<tf.Tensor: shape=(2, 10, 4), dtype=float32, numpy=
array([[[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)>

In [84]:
output_array

array([[[-0.14930329, -0.05488385, -0.03165275,  0.02511186],
        [-0.04538564, -0.02340828, -0.13736942,  0.03185076],
        [-0.18286747, -0.01420305,  0.02583414,  0.20739384],
        [-0.15128048, -0.15295134,  0.00826486, -0.03651741],
        [-0.19406578, -0.14820886, -0.24136184,  0.01954714],
        [-0.11608941,  0.05977409, -0.05805521,  0.08262399],
        [-0.10437049,  0.00281959,  0.00744648,  0.06939161],
        [-0.32889026, -0.11109954, -0.06513101,  0.02347144],
        [-0.10914627,  0.04005037, -0.11194003,  0.34588984],
        [-0.55321944, -0.24241613, -0.17234027,  0.2255739 ]],

       [[-0.12665638, -0.02560038, -0.07357763, -0.03386865],
        [-0.00296582,  0.05382628, -0.02431939, -0.00124017],
        [-0.152773  ,  0.01729362, -0.02049777,  0.00555693],
        [-0.01630396, -0.0508009 ,  0.00338947, -0.00536075],
        [-0.11710438, -0.0411608 ,  0.01888775,  0.10048383],
        [-0.1000828 ,  0.06160894, -0.04584875,  0.14583868],
      

In [67]:
output_softmax = tf.nn.softmax(output_array)
output_softmax

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0.24628553, 0.282701  , 0.2385324 , 0.23248109],
        [0.2298986 , 0.23856457, 0.25392184, 0.27761498],
        [0.2101039 , 0.2444843 , 0.26824066, 0.27717113],
        [0.22202027, 0.29728216, 0.2501223 , 0.23057525],
        [0.24084595, 0.2724257 , 0.25000373, 0.23672463]],

       [[0.23988546, 0.27957046, 0.23915865, 0.24138539],
        [0.24975686, 0.27135593, 0.24479878, 0.23408844],
        [0.24278015, 0.26340333, 0.24202275, 0.25179377],
        [0.23139507, 0.262904  , 0.25970972, 0.2459912 ],
        [0.2601803 , 0.25627998, 0.24524413, 0.23829558]]], dtype=float32)>

In [73]:
output_reshape = tf.reshape(output_softmax, [-1, 4])
output_reshape.shape

TensorShape([10, 4])

In [83]:
tf.reshape(tf.nn.softmax(output_array), [-1, 5, 4])

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0.24628553, 0.282701  , 0.2385324 , 0.23248109],
        [0.2298986 , 0.23856457, 0.25392184, 0.27761498],
        [0.2101039 , 0.2444843 , 0.26824066, 0.27717113],
        [0.22202027, 0.29728216, 0.2501223 , 0.23057525],
        [0.24084595, 0.2724257 , 0.25000373, 0.23672463]],

       [[0.23988546, 0.27957046, 0.23915865, 0.24138539],
        [0.24975686, 0.27135593, 0.24479878, 0.23408844],
        [0.24278015, 0.26340333, 0.24202275, 0.25179377],
        [0.23139507, 0.262904  , 0.25970972, 0.2459912 ],
        [0.2601803 , 0.25627998, 0.24524413, 0.23829558]]], dtype=float32)>

In [76]:
# tf.cast(tf.equal(y, tf.reduce_max(y,1,keep_dims=True)), y.dtype)
output_hard = tf.cast(tf.equal(output_reshape, tf.math.reduce_max(output_reshape, 1, keepdims=True)), output_softmax.dtype)
output_hard

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)>

In [81]:
tf.reshape(output_hard, [-1, 5, 4])

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]]], dtype=float32)>

In [322]:
tf.transpose(output_array, perm=[0, 2, 1]).shape

TensorShape([2, 4, 5])

In [316]:
layers.Softmax(output_array)

In [317]:
np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]).shape

(3, 3)

In [65]:
np.log(10)

2.302585092994046